<a href="https://colab.research.google.com/github/itsmoha/Unet-Model/blob/main/UnetModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from tensorflow.keras.layers import (
    Conv2D, MaxPool2D, BatchNormalization, Dropout, Input, Conv2DTranspose, Concatenate)
from tensorflow.keras.models import Model
import tensorflow as tf

In [8]:
def conv2D_block(x, filters=64, bn=False, apply_dropout=False, dpr=0.1):
    for _ in range(2):
        x =  Conv2D(filters, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(x)
        if bn:
            x = BatchNormalization()(x)
        if apply_dropout:
            x = Dropout(dpr)(x)
    return x

In [9]:
def unet(num_classes=2, input_shape=(512, 512, 1), num_layers=4, filters=64, out_activation='sigmoid'):

    inputs = Input(input_shape)
    x = inputs

    # encoder
    down_layers = []
    for nl in range(num_layers):
        x = conv2D_block(x, filters)
        down_layers.append(x)
        x = MaxPool2D((2, 2), strides=2)(x)
        filters *= 2

    #bottleneck
    x = Dropout(0.3)(x)
    x = conv2D_block(x, filters)

    # decoder
    for conv in reversed(down_layers):
        filters //= 2
        x = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(x)
        x = Concatenate(axis=3)([x, conv])

        x = conv2D_block(x, filters)

    # output
    outputs = Conv2D(num_classes, (1, 1), activation=out_activation)(x)

    return Model(inputs=[inputs], outputs=[outputs])
unet().summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________